In [45]:
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
from torch.utils.tensorboard import SummaryWriter
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [1]:
torch.manual_seed(0)
STATE_DIM = 3
dt = .1
T = 500 #Run simulation for 500 hours.
N = int(T / dt) + 1
T_span = np.linspace(0, T, N)
T_span_tensor = torch.Tensor(T_span)[(None,) * 2] #T_span needs to be converted to tensor object.

BATCH_SIZE = 1
device = torch.device("".join(["cuda:",f'{args.CUDA_ID}']) if torch.cuda.is_available() else "cpu")
LR = 1e-3
niter = 1000000

NameError: name 'torch' is not defined

In [47]:
obs_df_full = pd.read_csv('CON_synthetic_sol_df.csv') #Must be link to raw Github output if in Colab.
#obs_df_full = pd.read_csv('https://raw.githubusercontent.com/wallyxie/varInferenceSoilBiogeoModelSyntheticData/main/python_sde_variational_inference_code/CON_synthetic_sol_df.csv') #Must be link to raw Github output if in Colab.
obs_df = obs_df_full[obs_df_full['hour'] <= T] #Test with just first 1,000 hours of data.
obs_df

,hour,SOC,DOC,MBC
0,0,45.660340,0.071469,0.714689
1,10,45.646801,0.073782,0.713685
2,20,45.650479,0.073273,0.714127
3,30,45.644809,0.074233,0.713766
4,40,45.639221,0.075050,0.713440
5,50,45.642405,0.074620,0.713863
6,60,45.628300,0.076746,0.712869
7,70,45.633973,0.075864,0.713526
8,80,45.622501,0.077571,0.712763
9,90,45.621995,0.077491,0.712908


In [48]:
obs_times = torch.Tensor(np.array(obs_df['hour']))
obs_times

tensor([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110.,
        120., 130., 140., 150., 160., 170., 180., 190., 200., 210., 220., 230.,
        240., 250., 260., 270., 280., 290., 300., 310., 320., 330., 340., 350.,
        360., 370., 380., 390., 400., 410., 420., 430., 440., 450., 460., 470.,
        480., 490., 500.])

In [49]:
obs_means = torch.Tensor(np.array(obs_df.drop(columns = 'hour'))) #Convert to tensor.
obs_means

tensor([[45.6603,  0.0715,  0.7147],
        [45.6468,  0.0738,  0.7137],
        [45.6505,  0.0733,  0.7141],
        [45.6448,  0.0742,  0.7138],
        [45.6392,  0.0750,  0.7134],
        [45.6424,  0.0746,  0.7139],
        [45.6283,  0.0767,  0.7129],
        [45.6340,  0.0759,  0.7135],
        [45.6225,  0.0776,  0.7128],
        [45.6220,  0.0775,  0.7129],
        [45.6214,  0.0774,  0.7130],
        [45.6097,  0.0789,  0.7122],
        [45.6149,  0.0781,  0.7129],
        [45.6001,  0.0800,  0.7119],
        [45.6039,  0.0792,  0.7124],
        [45.5954,  0.0804,  0.7119],
        [45.5887,  0.0810,  0.7116],
        [45.5904,  0.0806,  0.7120],
        [45.5785,  0.0817,  0.7112],
        [45.5834,  0.0808,  0.7118],
        [45.5716,  0.0821,  0.7111],
        [45.5695,  0.0821,  0.7112],
        [45.5674,  0.0822,  0.7113],
        [45.5540,  0.0837,  0.7105],
        [45.5598,  0.0825,  0.7112],
        [45.5406,  0.0850,  0.7100],
        [45.5450,  0.0839,  0.7106],
 

In [50]:
obs_std = obs_means * 0.5 #Assume observations are distributed normally about the mean with standard deviation at half of the mean.
obs_std

tensor([[22.8302,  0.0357,  0.3573],
        [22.8234,  0.0369,  0.3568],
        [22.8252,  0.0366,  0.3571],
        [22.8224,  0.0371,  0.3569],
        [22.8196,  0.0375,  0.3567],
        [22.8212,  0.0373,  0.3569],
        [22.8141,  0.0384,  0.3564],
        [22.8170,  0.0379,  0.3568],
        [22.8113,  0.0388,  0.3564],
        [22.8110,  0.0387,  0.3565],
        [22.8107,  0.0387,  0.3565],
        [22.8049,  0.0394,  0.3561],
        [22.8074,  0.0390,  0.3564],
        [22.8000,  0.0400,  0.3559],
        [22.8019,  0.0396,  0.3562],
        [22.7977,  0.0402,  0.3560],
        [22.7944,  0.0405,  0.3558],
        [22.7952,  0.0403,  0.3560],
        [22.7892,  0.0409,  0.3556],
        [22.7917,  0.0404,  0.3559],
        [22.7858,  0.0411,  0.3555],
        [22.7847,  0.0411,  0.3556],
        [22.7837,  0.0411,  0.3556],
        [22.7770,  0.0419,  0.3552],
        [22.7799,  0.0413,  0.3556],
        [22.7703,  0.0425,  0.3550],
        [22.7725,  0.0420,  0.3553],
 

In [51]:
I_S = 0.001 + 0.0005 * torch.sin((2 * math.pi / (24 * 365)) * T_span_tensor) #Exogenous SOC input function
I_D =  0.0001 + 0.00005 * torch.sin((2 * math.pi / (24 * 365)) * T_span_tensor) #Exogenous DOC input function

In [52]:
temp_ref = 283

u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

scon_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M}

In [53]:
############################################################
##SOIL BIOGEOCHEMICAL MODEL TEMPERATURE RESPONSE FUNCTIONS##
############################################################

def temp_gen(t, temp_ref):
    temp = temp_ref + t / (20 * 24 * 365) + 10 * np.sin((2 * np.pi / 24) * t) + 10 * np.sin((2 * np.pi / (24 * 365)) * t)
    return temp

def arrhenius_temp_dep(parameter, temp, Ea, temp_ref):
    '''
    For a parameter with Arrhenius temperature dependence, returns the transformed parameter value.
    0.008314 is the gas constant. Temperatures are in K.
    '''
    decayed_parameter = parameter * np.exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref))
    return decayed_parameter

def linear_temp_dep(parameter, temp, Q, temp_ref):
    '''
    For a parameter with linear temperature dependence, returns the transformed parameter value.
    Q is the slope of the temperature dependence and is a varying parameter.
    Temperatures are in K.
    '''
    modified_parameter = parameter - Q * (temp - temp_ref)
    return modified_parameter

##########################################################################
##DETERMINISTIC SOIL BIOGEOCHEMICAL MODEL INITIAL STEADY STATE ESTIMATES##
##########################################################################

#Analytical_steady_state_init_awb to be coded later.
def analytical_steady_state_init_con(SOC_input, DOC_input, scon_params_dict):
    '''
    Returns a vector of C pool values to initialize an SCON system corresponding to set of parameter values using the analytical steady state equations of the deterministic CON system.
    Vector elements are in order of S_0, D_0, and M_0.
    Expected scon_params_dict = {scon_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M}    
    '''
    D_0 = (DOC_input + SOC_input * scon_params_dict['a_SD']) / (scon_params_dict['u_M'] + scon_params_dict['k_D_ref'] + scon_params_dict['u_M'] * scon_params_dict['a_M'] * (scon_params_dict['a_MSC'] - 1 - scon_params_dict['a_MSC'] * scon_params_dict['a_SD']) - scon_params_dict['a_DS'] * scon_params_dict['k_D_ref'] * scon_params_dict['a_SD'])
    S_0 = (SOC_input + D_0 * (scon_params_dict['a_DS'] * scon_params_dict['k_D_ref'] + scon_params_dict['u_M'] * scon_params_dict['a_M'] * scon_params_dict['a_MSC'])) / scon_params_dict['k_S_ref']
    M_0 = scon_params_dict['u_M'] * D_0 / scon_params_dict['k_M_ref']
    C_0_vector = torch.as_tensor([S_0, D_0, M_0])
    return C_0_vector

####################################################
##STOCHASTIC DIFFERENTIAL EQUATION MODEL FUNCTIONS##
#################################################### 

def drift_and_diffusion_scon(N, T_span_tensor, dt, I_S, I_D, analytical_steady_state_init_con, scon_params_dict, temp_ref, path_count):
    '''
    Returns SCON solutions using the Euler-Maruyama scheme.
    current_temp is output from temp_gen function. 
    Expected scon_params_dict = {scon_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M}
    '''
    system_var_size = 3 #SCON and CON will always have three state variables.
    C_vector = torch.zeros(path_count, system_var_size, N) #Create tensor for storing state variable values.
    drift_vector = C_vector.clone() #Create tensor to assign drift.
    C0 = analytical_steady_state_init_con(I_S[0, 0, 0].item(), I_D[0, 0, 0].item(), scon_params_dict)
    print('\n Initial pre-perturbation SOC, DOC, MBC = ', C0)
    C_vector[:, :, 0] = C0 #Assign deterministically generated initial conditions to all paths.
    diffusion_matrix_sqrt = torch.zeros([drift_vector.size(0), system_var_size, system_var_size, drift_vector.size(2)], device = drift_vector.device) #Create tensor to assign diffusion matrix elements.
    diffusion_matrix_sqrt[:, 0, 0, 0] = torch.sqrt(C0[0]) #Assigned S0 to element 1, 1 of matrix.
    diffusion_matrix_sqrt[:, 1, 1, 0] = torch.sqrt(C0[1]) #Assigned D0 to element 2, 2 of matrix.
    diffusion_matrix_sqrt[:, 2, 2, 0] = torch.sqrt(C0[2]) #Assigned M0 to element 3, 3 of matrix.
    current_temp = temp_gen(T_span_tensor, temp_ref) #Obtain temperature function vector across span of times.
    #Decay parameters are forced by temperature changes.
    k_S = arrhenius_temp_dep(scon_params_dict['k_S_ref'], current_temp, scon_params_dict['Ea_S'], temp_ref) #Apply vectorized temperature-dependent transformation to k_S_ref. 
    k_D = arrhenius_temp_dep(scon_params_dict['k_D_ref'], current_temp, scon_params_dict['Ea_D'], temp_ref) #Apply vectorized temperature-dependent transformation to k_D_ref.
    k_M = arrhenius_temp_dep(scon_params_dict['k_M_ref'], current_temp, scon_params_dict['Ea_M'], temp_ref) #Apply vectorized temperature-dependent transformation to k_M_ref.
    #print('\n k_S, k_D, k_M =', [k_S, k_D, k_M])
    #Drift vector is calculated (without litter input).
    for i in range(1, N):
        SOC, DOC, MBC = [C_vector[:, l : l + 1, i - 1] for l in range(system_var_size)]
        drift_SOC = I_S[0, 0, i - 1] + scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC + scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC - k_S[0, 0, i - 1] * SOC
        drift_DOC = I_D[0, 0, i - 1] + scon_params_dict['a_SD'] * k_S[0, 0, i - 1] * SOC + scon_params_dict['a_M'] * (1 - scon_params_dict['a_MSC']) * k_M[0, 0, i - 1] * MBC - (scon_params_dict['u_M'] + k_D[0, 0, i - 1]) * DOC
        drift_MBC = scon_params_dict['u_M'] * DOC - k_M[0, 0, i - 1] * MBC
        drift_vector[:, :, i - 1] = torch.cat([drift_SOC, drift_DOC, drift_MBC], 1) #Assign drift means to all paths.
        #Diffusion matrix is calculated.
        diffusion_matrix_sqrt[:, 0, 0, i - 1] = torch.sqrt(LowerBound.apply(drift_SOC, 0)).squeeze() #Assigned to element 1, 1 of matrix.
        diffusion_matrix_sqrt[:, 1, 1, i - 1] = torch.sqrt(LowerBound.apply(drift_DOC, 0)).squeeze() #Assigned to element 2, 2 of matrix.
        diffusion_matrix_sqrt[:, 2, 2, i - 1] = torch.sqrt(LowerBound.apply(drift_MBC, 0)).squeeze() #Assigned to element 3, 3 of matrix.
        C_vector[:, :, i] = d.multivariate_normal.MultivariateNormal(loc = C_vector[:, :, i - 1] + drift_vector[:, :, i - 1] * dt, scale_tril = diffusion_matrix_sqrt[:, :, :, i - 1] * math.sqrt(dt)).rsample()
        C_vector_test[:, :, i][C_vector_test[:, :, i] < 1e-9] = 1e-9 #Prevent state variables from dropping below 0.
    return C_vector, drift_vector, diffusion_matrix_sqrt

In [54]:
path_test = 2
C_vector_test = torch.zeros(path_test, 3, N)
drift_vector_test = torch.zeros(path_test, 3, N)
C0 = analytical_steady_state_init_con(I_S[0, 0, 0].item(), I_D[0, 0, 0].item(), scon_params_dict)
C_vector_test[:, :, 0] = C0
print(C_vector_test)
SOC, DOC, MBC = [C_vector_test[:, l : l + 1, 0] for l in range(3)]
print(SOC, DOC, MBC)

tensor([[[45.6603,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0715,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7147,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[45.6603,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0715,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7147,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
tensor([[45.6603],
        [45.6603]]) tensor([[0.0715],
        [0.0715]]) tensor([[0.7147],
        [0.7147]])


In [55]:
i = 1
SOC, DOC, MBC = [C_vector_test[:, l : l + 1, i - 1] for l in range(3)]
print(SOC, DOC, MBC)

tensor([[45.6603],
        [45.6603]]) tensor([[0.0715],
        [0.0715]]) tensor([[0.7147],
        [0.7147]])


In [56]:
print(scon_params_dict)

current_temp = temp_gen(T_span_tensor, temp_ref)

k_S = arrhenius_temp_dep(scon_params_dict['k_S_ref'], current_temp, scon_params_dict['Ea_S'], temp_ref)
k_D = arrhenius_temp_dep(scon_params_dict['k_D_ref'], current_temp, scon_params_dict['Ea_D'], temp_ref)
k_M = arrhenius_temp_dep(scon_params_dict['k_M_ref'], current_temp, scon_params_dict['Ea_M'], temp_ref)

print(k_S, k_D, k_M)
print(k_S[0, 0, 5])

{'u_M': 0.002, 'a_SD': 0.33, 'a_DS': 0.33, 'a_M': 0.33, 'a_MSC': 0.5, 'k_S_ref': 2.5e-05, 'k_D_ref': 0.005, 'k_M_ref': 0.0002, 'Ea_S': 75, 'Ea_D': 50, 'Ea_M': 50}
tensor([[[2.5000e-05, 2.5750e-05, 2.6519e-05,  ..., 1.3450e-05,
          1.3644e-05, 1.3851e-05]]]) tensor([[[0.0050, 0.0051, 0.0052,  ..., 0.0033, 0.0033, 0.0034]]]) tensor([[[0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001]]])
tensor(2.8952e-05)


In [57]:
class LowerBound(Function):
    @staticmethod
    def forward(ctx, inputs, bound):
        b = torch.ones(inputs.size()) * bound
        b = b.to(inputs.device)
        b = b.type(inputs.dtype)
        ctx.save_for_backward(inputs, b)
        return torch.max(inputs, b)
    @staticmethod
    def backward(ctx, grad_output):
        inputs, b = ctx.saved_tensors

        pass_through_1 = inputs >= b
        pass_through_2 = grad_output < 0

        pass_through = pass_through_1 | pass_through_2
        return pass_through.type(grad_output.dtype) * grad_output, None

class MaskedConv1d(nn.Conv1d):
    def __init__(self, mask_type, *args, **kwargs):
        super(MaskedConv1d, self).__init__(*args, **kwargs)
        assert mask_type in {'A', 'B'}
        self.register_buffer('mask', self.weight.data.clone())
        _, _, kW = self.weight.size()
        self.mask.fill_(1)
        self.mask[:, :, kW // 2 + 1*(mask_type == 'B'):] = 0

    def forward(self, x):
        self.weight.data *= self.mask
        return super(MaskedConv1d, self).forward(x)

class ResNetBlock(nn.Module):

    def __init__(self, inp_cha, out_cha, stride = 1, batch_norm=True):
        super().__init__()
        self.conv1 = MaskedConv1d('B', inp_cha,  out_cha, 15, stride, 7, bias=False)
        self.conv2 = MaskedConv1d('B', out_cha,  out_cha, 15, 1, 7, bias=False)

        self.act1 = nn.PReLU(out_cha, init = 0.2)
        self.act2 = nn.PReLU(out_cha, init = 0.2)

        if batch_norm:
            self.bn1 = nn.BatchNorm1d(out_cha)
            self.bn2 = nn.BatchNorm1d(out_cha)
        else:
            self.bn1 = nn.Identity()
            self.bn2 = nn.Identity()

        # If dimensions change, transform shortcut with a conv layer
        if inp_cha != out_cha or stride > 1:
            self.conv_skip = MaskedConv1d('B', inp_cha,  out_cha, 3, stride, 1, bias=False)
        else:
            self.conv_skip = nn.Identity()

    def forward(self, x):
        residual = x
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.act2(self.bn2(self.conv2(x) + self.conv_skip(residual)))
        return x

class ResNetBlockUnMasked(nn.Module):

    def __init__(self, inp_cha, out_cha, stride = 1, batch_norm=True):
        super().__init__()
        self.conv1 = nn.Conv1d(inp_cha,  out_cha, 15, stride, 7)
        self.conv2 = nn.Conv1d(out_cha,  out_cha, 15, 1, 7)

        self.act1 = nn.PReLU(out_cha, init = 0.2)
        self.act2 = nn.PReLU(out_cha, init = 0.2)

        if batch_norm:
            self.bn1 = nn.BatchNorm1d(out_cha)
            self.bn2 = nn.BatchNorm1d(out_cha)
        else:
            self.bn1 = nn.Identity()
            self.bn2 = nn.Identity()

        # If dimensions change, transform shortcut with a conv layer
        if inp_cha != out_cha or stride > 1:
            self.conv_skip = nn.Conv1d(inp_cha,  out_cha, 15, stride, 7, bias=False)
        else:
            self.conv_skip = nn.Identity()

    def forward(self, x):
        residual = x
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.act2(self.bn2(self.conv2(x) + self.conv_skip(residual)))
        return x

class CouplingLayer(nn.Module):

    def __init__(self, cond_inputs, stride):
        super().__init__()
        self.net = nn.Sequential(ResNetBlock(1+cond_inputs, 96),
                                 MaskedConv1d('B', 96,  2, 15, stride, 7, bias=False))

        self.feature_net = nn.Sequential(ResNetBlockUnMasked(cond_inputs, 96),
                                          ResNetBlockUnMasked(96, cond_inputs))

        self.unpack = True if cond_inputs > 1 else False

    def forward(self, x, cond_inputs):
        if self.unpack:
            cond_inputs = torch.cat([*cond_inputs], 1)
        cond_inputs = self.feature_net(cond_inputs)
        feature_vec = torch.cat([x, cond_inputs], 1)
        output = self.net(feature_vec)
        mu, sigma = torch.chunk(output, 2, 1)
        mu = self._pass_through_units(mu)
        sigma = self._pass_through_units(sigma, mu=False)
        x = mu + sigma*x
        return x, sigma

    def _pass_through_units(self, params, mu=True):
        B, _, L = params.shape
        padding = 2 if STATE_DIM % 2==1 else 1
        if mu:
            pad = torch.zeros([B, padding, L], device=params.device)
        else:
            params = LowerBound.apply(params, 1e-6)
            pad = torch.ones([B, padding, L], device=params.device)
        return torch.cat([pad, params], 1).transpose(2, 1).reshape(B, 1, -1)

class PermutationLayer(nn.Module):

    def __init__(self):
        super().__init__()
        self.index = torch.randperm(STATE_DIM)

    def forward(self, x):
        B, S, L = x.shape
        x_reshape = x.reshape(B, S, -1, STATE_DIM)
        x_perm = x_reshape[:, :, :, self.index]
        x = x_perm.reshape(B, S, L)
        return x

In [58]:
path_test = 2
C_vector_test = torch.zeros(path_test, 3, N)
C0 = analytical_steady_state_init_con(I_S[0, 0, 0].item(), I_D[0, 0, 0].item(), scon_params_dict)
C_vector_test[:, :, 0] = C0

system_var_size = 3

#diffusion_matrix_sqrt = torch.zeros([drift_vector_test.size(0), drift_vector_test.size(2), system_var_size, system_var_size], device = C_vector_test.device) #Create 3 x 3 zeros tensor to assign diffusion matrix elements.
diffusion_matrix_sqrt_test = torch.zeros([C_vector_test.size(0), system_var_size, system_var_size, C_vector_test.size(2)]) #Create 3 x 3 zeros tensor to assign diffusion matrix elements.
diffusion_matrix_sqrt_test[:, 0, 0, 0] = torch.sqrt(C0[0]) #Assigned S0 to element 1, 1 of matrix.
diffusion_matrix_sqrt_test[:, 1, 1, 0] = torch.sqrt(C0[1]) #Assigned D0 to element 2, 2 of matrix.
diffusion_matrix_sqrt_test[:, 2, 2, 0] = torch.sqrt(C0[2]) #Assigned M0 to element 3, 3 of matrix.

drift_vector_test = torch.zeros(path_test, 3, N)

#First step from 0 to dt trial
i = 1
SOC, DOC, MBC = [C_vector_test[:, l : l + 1, i - 1] for l in range(system_var_size)]
print('\n C_vector_test:', C_vector_test)
print('\n C0:', SOC, DOC, MBC)
print('\n k:', k_D[0, 0, i - 1], k_S[0, 0, i - 1], k_M[0, 0, i - 1])
print('\n I_S:', I_S[0, 0, i - 1])
print("\n scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC:", scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC)
print("\n scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC:", scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC)
print('\n -k_S[0, 0, i - 1] * SOC:', -k_S[0, 0, i - 1] * SOC)
print("\n I_S[0, 0, i - 1] + scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC + scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC - k_S[0, 0, i - 1] * SOC:", I_S[0, 0, i - 1] + scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC + scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC - k_S[0, 0, i - 1] * SOC)
drift_SOC = I_S[0, 0, i - 1] + scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC + scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC - k_S[0, 0, i - 1] * SOC
drift_DOC = I_D[0, 0, i - 1] + scon_params_dict['a_SD'] * k_S[0, 0, i - 1] * SOC + scon_params_dict['a_M'] * (1 - scon_params_dict['a_MSC']) * k_M[0, 0, i - 1] * MBC - (scon_params_dict['u_M'] + k_D[0, 0, i - 1]) * DOC
print("\n scon_params_dict['u_M'] * DOC:", scon_params_dict['u_M'] * DOC)
print("\n -k_M[0, 0, i - 1] * MBC:", -k_M[0, 0, i - 1] * MBC)
drift_MBC = scon_params_dict['u_M'] * DOC - k_M[0, 0, i - 1] * MBC
print('\n drift_SOC:', drift_SOC)
print('\n drift_DOC:', drift_DOC)
print('\n drift_MBC:', drift_MBC)
print('\n torch.cat([drift_SOC, drift_DOC, drift_MBC], 1):', torch.cat([drift_SOC, drift_DOC, drift_MBC], 1))
#drift_vector_test[:, :, i - 1] = torch.cat([drift_SOC, drift_DOC, drift_MBC], 1) #Assign drift to all paths.
drift_vector_test[:, 0 : 1, i - 1] = drift_SOC
drift_vector_test[:, 1 : 2, i - 1] = drift_DOC
drift_vector_test[:, 2 : 3, i - 1] = drift_MBC
#Diffusion matrix is calculated (recall litter input is not a part of the drift vector or diffusion matrix).
diffusion_matrix_sqrt_test[:, 0, 0, i - 1] = torch.sqrt(LowerBound.apply(drift_SOC, 1e-10)).squeeze() #Assigned to element 1, 1 of matrix.
diffusion_matrix_sqrt_test[:, 1, 1, i - 1] = torch.sqrt(LowerBound.apply(drift_DOC, 1e-10)).squeeze() #Assigned to element 2, 2 of matrix.
diffusion_matrix_sqrt_test[:, 2, 2, i - 1] = torch.sqrt(LowerBound.apply(drift_MBC, 1e-10)).squeeze() #Assigned to element 3, 3 of matrix.

print('\n Check initial conditions not erased:', C_vector_test[:, :, 0])
print('\n Check drift vector:', drift_vector_test[:, :, i - 1] * dt)
update_test = C_vector_test[:, :, i - 1] + drift_vector_test[:, :, i - 1] * dt
print('\n Check diffusion matrix:', diffusion_matrix_sqrt_test[:, :, :, i - 1])
print('\n C_vector_test[0, :, i - 1]:', C_vector_test[0, :, i - 1])
print('\n drift_vector_test[:, :, i - 1] * dt:', drift_vector_test[0, :, i - 1] * dt)
print('\n Check Euler step without noise:', update_test)
C_vector_test[:, :, i] = d.multivariate_normal.MultivariateNormal(loc = C_vector_test[:, :, i - 1] + drift_vector_test[:, :, i - 1] * dt, scale_tril = diffusion_matrix_sqrt_test[:, :, :, i - 1] * math.sqrt(dt)).rsample()
#Error message for the above. Probably need to permute somewhere.
#for k in range(path_test):
    #C_vector_test[k, :, i] = d.multivariate_normal.MultivariateNormal(loc = C_vector_test[k, :, i - 1] + drift_vector_test[k, :, i - 1] * dt, scale_tril = diffusion_matrix_sqrt[k, :, :, i - 1] * math.sqrt(dt))


 C_vector_test: tensor([[[45.6603,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0715,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7147,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[45.6603,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0715,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.7147,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

 C0: tensor([[45.6603],
        [45.6603]]) tensor([[0.0715],
        [0.0715]]) tensor([[0.7147],
        [0.7147]])

 k: tensor(0.0050) tensor(2.5000e-05) tensor(0.0002)

 I_S: tensor(0.0010)

 scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC: tensor([[0.0001],
        [0.0001]])

 scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC: tensor([[2.3585e-05],
        [2.3585e-05]])

 -k_S[0, 0, i - 1] * SOC: tensor([[-0.0011],
        [-0.0011]])

 I_S[0, 0, i - 1] + scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC + scon

In [59]:
C_vector_test = torch.zeros(path_test, 3, N)
C0 = analytical_steady_state_init_con(I_S[0, 0, 0].item(), I_D[0, 0, 0].item(), scon_params_dict)
C_vector_test[:, :, 0] = C0

system_var_size = 3

diffusion_matrix_sqrt_test = torch.zeros([C_vector_test.size(0), system_var_size, system_var_size, C_vector_test.size(2)]) #Create 3 x 3 zeros tensor to assign diffusion matrix elements.
diffusion_matrix_sqrt_test[:, 0, 0, 0] = torch.sqrt(C0[0]) #Assigned S0 to element 1, 1 of matrix.
diffusion_matrix_sqrt_test[:, 1, 1, 0] = torch.sqrt(C0[1]) #Assigned D0 to element 2, 2 of matrix.
diffusion_matrix_sqrt_test[:, 2, 2, 0] = torch.sqrt(C0[2]) #Assigned M0 to element 3, 3 of matrix.

drift_vector_test = torch.zeros(path_test, 3, N)


#Euler-Maruyama loop trial
#for i in range(1, len(T_span_tensor[0, 0, :])):
#for i in range(1, 2):
for i in range(1, N):
    SOC, DOC, MBC = [C_vector_test[:, l : l + 1, i - 1] for l in range(system_var_size)]
    print('SOC, DOC, MBC = ', SOC, ",", DOC, ",", MBC)
    drift_SOC = I_S[0, 0, i - 1] + scon_params_dict['a_DS'] * k_D[0, 0, i - 1] * DOC + scon_params_dict['a_M'] * scon_params_dict['a_MSC'] * k_M[0, 0, i - 1] * MBC - k_S[0, 0, i - 1] * SOC
    drift_DOC = I_D[0, 0, i - 1] + scon_params_dict['a_SD'] * k_S[0, 0, i - 1] * SOC + scon_params_dict['a_M'] * (1 - scon_params_dict['a_MSC']) * k_M[0, 0, i - 1] * MBC - (scon_params_dict['u_M'] + k_D[0, 0, i - 1]) * DOC
    drift_MBC = scon_params_dict['u_M'] * DOC - k_M[0, 0, i - 1] * MBC
    #drift_vector_test[:, :, i - 1] = torch.cat([drift_SOC, drift_DOC, drift_MBC], 1) #Assign drift to all paths.
    drift_vector_test[:, 0 : 1, i - 1] = drift_SOC
    drift_vector_test[:, 1 : 2, i - 1] = drift_DOC
    drift_vector_test[:, 2 : 3, i - 1] = drift_MBC    #Diffusion matrix is calculated (recall litter input is not a part of the drift vector or diffusion matrix).
    diffusion_matrix_sqrt_test[:, 0, 0, i - 1] = torch.sqrt(LowerBound.apply(drift_SOC, 1e-10)).squeeze() #Assigned to element 1, 1 of matrix.
    diffusion_matrix_sqrt_test[:, 1, 1, i - 1] = torch.sqrt(LowerBound.apply(drift_DOC, 1e-10)).squeeze() #Assigned to element 2, 2 of matrix.
    diffusion_matrix_sqrt_test[:, 2, 2, i - 1] = torch.sqrt(LowerBound.apply(drift_MBC, 1e-10)).squeeze() #Assigned to element 3, 3 of matrix.
    C_vector_test[:, :, i] = d.multivariate_normal.MultivariateNormal(loc = C_vector_test[:, :, i - 1] + drift_vector_test[:, :, i - 1] * dt, scale_tril = diffusion_matrix_sqrt_test[:, :, :, i - 1] * math.sqrt(dt)).rsample()
    C_vector_test[:, :, i][C_vector_test[:, :, i] < 1e-9] = 1e-9
print(C_vector_test)

SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0715],
        [0.0715]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0715],
        [0.0715]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0722],
        [0.0711]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0713],
        [0.0721]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0729],
        [0.0721]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0716],
        [0.0694]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tensor([[0.0708],
        [0.0693]]) , tensor([[0.7147],
        [0.7147]])
SOC, DOC, MBC =  tensor([[45.6603],
        [45.6603]]) , tens

SOC, DOC, MBC =  tensor([[45.6479],
        [45.6468]]) , tensor([[0.1026],
        [0.0280]]) , tensor([[0.7141],
        [0.7134]])
SOC, DOC, MBC =  tensor([[45.6478],
        [45.6466]]) , tensor([[0.1026],
        [0.0399]]) , tensor([[0.7141],
        [0.7134]])
SOC, DOC, MBC =  tensor([[45.6477],
        [45.6465]]) , tensor([[0.1026],
        [0.0478]]) , tensor([[0.7141],
        [0.7134]])
SOC, DOC, MBC =  tensor([[45.6476],
        [45.6465]]) , tensor([[0.1026],
        [0.0432]]) , tensor([[0.7141],
        [0.7134]])
SOC, DOC, MBC =  tensor([[45.6475],
        [45.6463]]) , tensor([[0.1025],
        [0.0417]]) , tensor([[0.7141],
        [0.7134]])
SOC, DOC, MBC =  tensor([[45.6474],
        [45.6463]]) , tensor([[0.1025],
        [0.0390]]) , tensor([[0.7141],
        [0.7133]])
SOC, DOC, MBC =  tensor([[45.6474],
        [45.6462]]) , tensor([[0.1025],
        [0.0427]]) , tensor([[0.7141],
        [0.7133]])
SOC, DOC, MBC =  tensor([[45.6473],
        [45.6461]]) , tens

SOC, DOC, MBC =  tensor([[45.6837],
        [45.6792]]) , tensor([[0.1006],
        [0.0667]]) , tensor([[0.6955],
        [0.7261]])
SOC, DOC, MBC =  tensor([[45.6921],
        [45.6851]]) , tensor([[0.1006],
        [0.0667]]) , tensor([[0.6985],
        [0.7267]])
SOC, DOC, MBC =  tensor([[45.6942],
        [45.6826]]) , tensor([[0.1006],
        [0.0667]]) , tensor([[0.6890],
        [0.7249]])
SOC, DOC, MBC =  tensor([[45.6897],
        [45.6760]]) , tensor([[0.1005],
        [0.0667]]) , tensor([[0.6924],
        [0.7244]])
SOC, DOC, MBC =  tensor([[45.6913],
        [45.6698]]) , tensor([[0.1005],
        [0.0667]]) , tensor([[0.6883],
        [0.7254]])
SOC, DOC, MBC =  tensor([[45.6922],
        [45.6676]]) , tensor([[0.1005],
        [0.0667]]) , tensor([[0.6835],
        [0.7238]])
SOC, DOC, MBC =  tensor([[45.6848],
        [45.6694]]) , tensor([[0.1005],
        [0.0667]]) , tensor([[0.6783],
        [0.7221]])
SOC, DOC, MBC =  tensor([[45.6997],
        [45.6691]]) , tens

        [45.6885]]) , tensor([[0.0993],
        [0.0341]]) , tensor([[0.7070],
        [0.7113]])
SOC, DOC, MBC =  tensor([[45.6752],
        [45.6885]]) , tensor([[0.0993],
        [0.0425]]) , tensor([[0.7070],
        [0.7113]])
SOC, DOC, MBC =  tensor([[45.6751],
        [45.6884]]) , tensor([[0.0993],
        [0.0440]]) , tensor([[0.7070],
        [0.7113]])
SOC, DOC, MBC =  tensor([[45.6750],
        [45.6883]]) , tensor([[0.0992],
        [0.0454]]) , tensor([[0.7070],
        [0.7113]])
SOC, DOC, MBC =  tensor([[45.6750],
        [45.6882]]) , tensor([[0.0992],
        [0.0456]]) , tensor([[0.7070],
        [0.7113]])
SOC, DOC, MBC =  tensor([[45.6749],
        [45.6881]]) , tensor([[0.0992],
        [0.0529]]) , tensor([[0.7070],
        [0.7112]])
SOC, DOC, MBC =  tensor([[45.6748],
        [45.6880]]) , tensor([[0.0992],
        [0.0527]]) , tensor([[0.7070],
        [0.7112]])
SOC, DOC, MBC =  tensor([[45.6747],
        [45.6879]]) , tensor([[0.0992],
        [0.0518]]) , t

SOC, DOC, MBC =  tensor([[45.6621],
        [45.6750]]) , tensor([[0.0887],
        [0.1006]]) , tensor([[0.7063],
        [0.7112]])
SOC, DOC, MBC =  tensor([[45.6621],
        [45.6750]]) , tensor([[0.0887],
        [0.1005]]) , tensor([[0.7063],
        [0.7135]])
SOC, DOC, MBC =  tensor([[45.6620],
        [45.6749]]) , tensor([[0.0887],
        [0.1005]]) , tensor([[0.7063],
        [0.7124]])
SOC, DOC, MBC =  tensor([[45.6620],
        [45.6749]]) , tensor([[0.0886],
        [0.1005]]) , tensor([[0.7063],
        [0.7123]])
SOC, DOC, MBC =  tensor([[45.6619],
        [45.6748]]) , tensor([[0.0886],
        [0.1005]]) , tensor([[0.7063],
        [0.7123]])
SOC, DOC, MBC =  tensor([[45.6619],
        [45.6748]]) , tensor([[0.0886],
        [0.1005]]) , tensor([[0.7063],
        [0.7138]])
SOC, DOC, MBC =  tensor([[45.6619],
        [45.6748]]) , tensor([[0.0886],
        [0.1005]]) , tensor([[0.7065],
        [0.7128]])
SOC, DOC, MBC =  tensor([[45.6618],
        [45.6747]]) , tens

...

        [45.6437]]) , tensor([[0.1133],
        [0.1156]]) , tensor([[0.8479],
        [0.5034]])
SOC, DOC, MBC =  tensor([[45.4925],
        [45.6436]]) , tensor([[0.1133],
        [0.1156]]) , tensor([[0.8479],
        [0.5051]])
SOC, DOC, MBC =  tensor([[45.4924],
        [45.6435]]) , tensor([[0.1133],
        [0.1155]]) , tensor([[0.8479],
        [0.5035]])
SOC, DOC, MBC =  tensor([[45.4923],
        [45.6434]]) , tensor([[0.1132],
        [0.1155]]) , tensor([[0.8479],
        [0.5004]])
SOC, DOC, MBC =  tensor([[45.4922],
        [45.6433]]) , tensor([[0.1132],
        [0.1155]]) , tensor([[0.8479],
        [0.4978]])
SOC, DOC, MBC =  tensor([[45.4921],
        [45.6432]]) , tensor([[0.1132],
        [0.1155]]) , tensor([[0.8479],
        [0.4995]])
SOC, DOC, MBC =  tensor([[45.4920],
        [45.6431]]) , tensor([[0.1132],
        [0.1154]]) , tensor([[0.8479],
        [0.5002]])
SOC, DOC, MBC =  tensor([[45.4919],
        [45.6430]]) , tensor([[0.1132],
        [0.1154]]) , t

        [45.6388]]) , tensor([[0.1117],
        [0.1139]]) , tensor([[0.8834],
        [0.5073]])
SOC, DOC, MBC =  tensor([[45.4225],
        [45.6179]]) , tensor([[0.1117],
        [0.1138]]) , tensor([[0.8846],
        [0.5085]])
SOC, DOC, MBC =  tensor([[45.4257],
        [45.6335]]) , tensor([[0.1117],
        [0.1138]]) , tensor([[0.8868],
        [0.5023]])
SOC, DOC, MBC =  tensor([[45.4337],
        [45.6286]]) , tensor([[0.1117],
        [0.1138]]) , tensor([[0.8842],
        [0.5043]])
SOC, DOC, MBC =  tensor([[45.4458],
        [45.6023]]) , tensor([[0.1116],
        [0.1138]]) , tensor([[0.8834],
        [0.5003]])
SOC, DOC, MBC =  tensor([[45.4386],
        [45.5949]]) , tensor([[0.1116],
        [0.1137]]) , tensor([[0.8795],
        [0.4992]])
SOC, DOC, MBC =  tensor([[45.4520],
        [45.5900]]) , tensor([[0.1116],
        [0.1137]]) , tensor([[0.8857],
        [0.4961]])
SOC, DOC, MBC =  tensor([[45.4485],
        [45.5935]]) , tensor([[0.1116],
        [0.1137]]) , t

SOC, DOC, MBC =  tensor([[45.4490],
        [45.5166]]) , tensor([[0.1098],
        [0.1118]]) , tensor([[0.8572],
        [0.5596]])
SOC, DOC, MBC =  tensor([[45.4490],
        [45.5166]]) , tensor([[0.1098],
        [0.1118]]) , tensor([[0.8570],
        [0.5628]])
SOC, DOC, MBC =  tensor([[45.4490],
        [45.5166]]) , tensor([[0.1098],
        [0.1118]]) , tensor([[0.8570],
        [0.5647]])
SOC, DOC, MBC =  tensor([[45.4489],
        [45.5166]]) , tensor([[0.1098],
        [0.1118]]) , tensor([[0.8570],
        [0.5656]])
SOC, DOC, MBC =  tensor([[45.4489],
        [45.5165]]) , tensor([[0.1097],
        [0.1117]]) , tensor([[0.8570],
        [0.5655]])
SOC, DOC, MBC =  tensor([[45.4489],
        [45.5165]]) , tensor([[0.1097],
        [0.1117]]) , tensor([[0.8570],
        [0.5640]])
SOC, DOC, MBC =  tensor([[45.4488],
        [45.5165]]) , tensor([[0.1097],
        [0.1117]]) , tensor([[0.8570],
        [0.5607]])
SOC, DOC, MBC =  tensor([[45.4488],
        [45.5164]]) , tens

SOC, DOC, MBC =  tensor([[45.4342],
        [45.5020]]) , tensor([[0.0695],
        [0.1174]]) , tensor([[0.8558],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4339],
        [45.5017]]) , tensor([[0.0677],
        [0.1174]]) , tensor([[0.8558],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4336],
        [45.5014]]) , tensor([[0.0844],
        [0.1174]]) , tensor([[0.8558],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4333],
        [45.5011]]) , tensor([[0.0875],
        [0.1174]]) , tensor([[0.8558],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4330],
        [45.5009]]) , tensor([[0.0888],
        [0.1174]]) , tensor([[0.8557],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4327],
        [45.5006]]) , tensor([[0.0920],
        [0.1174]]) , tensor([[0.8557],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4325],
        [45.5003]]) , tensor([[0.0942],
        [0.1174]]) , tensor([[0.8557],
        [0.5642]])
SOC, DOC, MBC =  tensor([[45.4322],
        [45.5001]]) , tens

        [45.4757]]) , tensor([[0.1091],
        [0.1159]]) , tensor([[0.8565],
        [0.5595]])
SOC, DOC, MBC =  tensor([[45.4316],
        [45.4741]]) , tensor([[0.1091],
        [0.1158]]) , tensor([[0.8576],
        [0.5556]])
SOC, DOC, MBC =  tensor([[45.4286],
        [45.4836]]) , tensor([[0.1091],
        [0.1158]]) , tensor([[0.8581],
        [0.5538]])
SOC, DOC, MBC =  tensor([[45.4321],
        [45.4769]]) , tensor([[0.1090],
        [0.1158]]) , tensor([[0.8525],
        [0.5453]])
SOC, DOC, MBC =  tensor([[45.4372],
        [45.4700]]) , tensor([[0.1090],
        [0.1157]]) , tensor([[0.8537],
        [0.5447]])
SOC, DOC, MBC =  tensor([[45.4376],
        [45.4656]]) , tensor([[0.1090],
        [0.1157]]) , tensor([[0.8569],
        [0.5447]])
SOC, DOC, MBC =  tensor([[45.4456],
        [45.4665]]) , tensor([[0.1089],
        [0.1157]]) , tensor([[0.8537],
        [0.5441]])
SOC, DOC, MBC =  tensor([[45.4389],
        [45.4622]]) , tensor([[0.1089],
        [0.1156]]) , t

In [60]:
print(C_vector_test[:, :, 0].size())
print(drift_vector_test[:, :, 0].size())
print(C_vector_test[:, :, 0])
print(drift_vector_test[:, :, 0])

torch.Size([2, 3])
torch.Size([2, 3])
tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])
tensor([[-1.1642e-10,  0.0000e+00,  0.0000e+00],
        [-1.1642e-10,  0.0000e+00,  0.0000e+00]])


In [198]:
print(C_vector_test[:, :, 10])
print(drift_vector_test[:, :, 10])
print(diffusion_matrix_sqrt_test[:, :, :, 10])

tensor([[ 4.5654e+01, -1.1616e-02,  7.1385e-01],
        [ 4.5660e+01, -1.0476e-01,  7.1448e-01]])
tensor([[-0.0005, -0.0109, -0.0002],
        [-0.0007, -0.1033, -0.0004]])
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])


In [24]:
drift_and_diffusion_scon(10, T_span_tensor, dt, I_S, I_D, analytical_steady_state_init_con, scon_params_dict, temp_ref, path_test)


 Initial pre-perturbation SOC, DOC, MBC =  tensor([45.6603,  0.0715,  0.7147])


(tensor([[[45.6603, 45.6603, 45.6603, 45.6603, 45.6603, 45.6603, 45.6603,
           45.6603, 45.6603, 45.6602],
          [ 0.0715,  0.0715,  0.0705,  0.0700,  0.0708,  0.0707,  0.0704,
            0.0687,  0.0730,  0.0725],
          [ 0.7147,  0.7147,  0.7147,  0.7147,  0.7147,  0.7147,  0.7147,
            0.7147,  0.7147,  0.7147]],
 
         [[45.6603, 45.6603, 45.6603, 45.6603, 45.6603, 45.6603, 45.6603,
           45.6603, 45.6603, 45.6602],
          [ 0.0715,  0.0715,  0.0715,  0.0715,  0.0734,  0.0728,  0.0725,
            0.0740,  0.0749,  0.0728],
          [ 0.7147,  0.7147,  0.7147,  0.7147,  0.7147,  0.7147,  0.7147,
            0.7147,  0.7147,  0.7147]]]),
 tensor([[[-1.1642e-10, -3.1375e-05, -6.5296e-05, -9.9348e-05, -1.3197e-04,
           -1.6710e-04, -2.0344e-04, -2.4333e-04, -2.7268e-04,  0.0000e+00],
          [ 0.0000e+00,  4.6595e-06,  1.6510e-05,  2.5313e-05,  2.4672e-05,
            3.1017e-05,  3.8984e-05,  5.8079e-05,  3.0927e-05,  0.0000e+00],
          

In [200]:
drift_and_diffusion_scon(N, T_span_tensor, dt, I_S, I_D, analytical_steady_state_init_con, scon_params_dict, temp_ref, path_test)


 Initial pre-perturbation SOC, DOC, MBC =  tensor([45.6603,  0.0715,  0.7147])


(tensor([[[45.6603, 45.6603, 45.6604,  ...,     nan,     nan,     nan],
          [ 0.0715,  0.1609,  0.0507,  ...,     nan,     nan,     nan],
          [ 0.7147,  0.7147,  0.7143,  ...,     nan,     nan,     nan]],
 
         [[45.6603, 45.6603, 45.6673,  ...,     nan,     nan,     nan],
          [ 0.0715,  0.2821,  0.6116,  ...,     nan,     nan,     nan],
          [ 0.7147,  0.7147,  0.7123,  ...,     nan,     nan,     nan]]]),
 tensor([[[-1.1642e-10,  1.1914e-04, -9.9310e-05,  ...,         nan,
                   nan,  0.0000e+00],
          [ 7.1462e-02,  1.6027e-01,  5.0839e-02,  ...,         nan,
                   nan,  0.0000e+00],
          [ 0.0000e+00,  1.7604e-04, -4.7223e-05,  ...,         nan,
                   nan,  0.0000e+00]],
 
         [[-1.1642e-10,  3.2306e-04,  8.6309e-04,  ...,         nan,
                   nan,  0.0000e+00],
          [ 7.1462e-02,  2.8057e-01,  6.0766e-01,  ...,         nan,
                   nan,  0.0000e+00],
          [ 0.0000e+00, 

In [ ]:
class SDEFlow(nn.Module):

    def __init__(self, cond_inputs=1):
        super().__init__()

        stride = 3 if STATE_DIM % 2==1 else 2

        self.CL_1 = CouplingLayer(cond_inputs, stride)
        self.CL_2 = CouplingLayer(cond_inputs, stride)
        self.CL_3 = CouplingLayer(cond_inputs, stride)
        self.CL_4 = CouplingLayer(cond_inputs, stride)
        self.CL_5 = CouplingLayer(cond_inputs, stride)

        self.P_1 = PermutationLayer()
        self.P_2 = PermutationLayer()
        self.P_3 = PermutationLayer()
        self.P_4 = PermutationLayer()

        self.base_dist = d.normal.Normal(loc = 0., scale = 1.0)

    def forward(self, batch_size, *args, **kwargs):

        eps = self.base_dist.sample([batch_size, 1, STATE_DIM*N]).to(device)
        log_prob = self.base_dist.log_prob(eps).sum(-1)

        times = torch.arange(0, T+dt, dt, device=eps.device)[(None,)*2].repeat(batch_size, STATE_DIM, 1).transpose(-2, -1).reshape(batch_size, 1, -1)

        CL_1, CL_1_sigma = self.CL_1(eps, times)
        P_1 = self.P_1(CL_1)

        CL_2, CL_2_sigma = self.CL_2(P_1, times)
        P_2 = self.P_2(CL_2)

        CL_3, CL_3_sigma = self.CL_3(P_2, times)
        P_3 = self.P_3(CL_3)

        CL_4, CL_4_sigma = self.CL_4(P_3, times)
        P_4 = self.P_4(CL_4)

        y, CL_5_sigma = self.CL_5(P_4, times)

        for sigma in [CL_1_sigma, CL_2_sigma, CL_3_sigma, CL_4_sigma, CL_5_sigma]:
            log_prob -= torch.log(sigma).sum(-1)

        return y.reshape(batch_size, STATE_DIM, -1), log_prob

class ObsModel(nn.Module):

    def __init__(self, times, mu, scale):
        super().__init__()

        self.idx = self._get_idx(times)
        self.times = times
        self.mu = torch.Tensor(mu).to(device)
        self.scale = scale
        
    def forward(self, x):
            obs_ll = d.normal.Normal(self.mu.permute(1, 0), self.scale).log_prob(x[:, self.idx, :])
            return torch.sum(obs_ll, [-1, -2]).mean()

    def _get_idx(self, times):
        return list((times/dt).astype(int))  